In [1]:
pip install CTGANENN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.8/169.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu1

In [2]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dataset = pd.read_csv('https://raw.githubusercontent.com/mahayasa/gan-hybrid-sampling-customer-churn/main/data/telco_new.csv')

df1=dataset[dataset['Churn']==0]
df=dataset[dataset['Churn']==1]

In [3]:
from ctganenn import CTGANENN

X,y=CTGANENN(df,df1,2655,'Churn')

Gen. (-1.07) | Discrim. (-0.26): 100%|██████████| 100/100 [01:02<00:00,  1.59it/s]


In [10]:
# define model
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
#balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975652 using {'class_weight': {0: 80, 1: 100}}
0.954041 (0.007254) with: {'class_weight': {0: 10, 1: 100}}
0.958498 (0.008243) with: {'class_weight': {0: 20, 1: 100}}
0.962639 (0.007732) with: {'class_weight': {0: 30, 1: 100}}
0.965582 (0.006251) with: {'class_weight': {0: 40, 1: 100}}
0.969383 (0.005554) with: {'class_weight': {0: 50, 1: 100}}
0.971877 (0.006159) with: {'class_weight': {0: 60, 1: 100}}
0.974996 (0.004641) with: {'class_weight': {0: 70, 1: 100}}
0.975652 (0.005418) with: {'class_weight': {0: 80, 1: 100}}
0.974784 (0.006914) with: {'class_weight': {0: 90, 1: 100}}
0.975163 (0.005335) with: {'class_weight': {0: 100, 1: 100}}


In [11]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('DT-telco3-ctgan-auc-fn.xlsx', index=False)

In [5]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.977154 using {'class_weight': {0: 100, 1: 100}}
0.955180 (0.007201) with: {'class_weight': {0: 10, 1: 100}}
0.959374 (0.007029) with: {'class_weight': {0: 20, 1: 100}}
0.962565 (0.008361) with: {'class_weight': {0: 30, 1: 100}}
0.965681 (0.005426) with: {'class_weight': {0: 40, 1: 100}}
0.971661 (0.005529) with: {'class_weight': {0: 50, 1: 100}}
0.973956 (0.005284) with: {'class_weight': {0: 60, 1: 100}}
0.974714 (0.005228) with: {'class_weight': {0: 70, 1: 100}}
0.976590 (0.005084) with: {'class_weight': {0: 80, 1: 100}}
0.974857 (0.005058) with: {'class_weight': {0: 90, 1: 100}}
0.977154 (0.006405) with: {'class_weight': {0: 100, 1: 100}}


In [6]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('DT-telco3-ctgan-f1-fn.xlsx', index=False)

In [7]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.975920 using {'class_weight': {0: 100, 1: 100}}
0.955238 (0.005475) with: {'class_weight': {0: 10, 1: 100}}
0.958936 (0.006831) with: {'class_weight': {0: 20, 1: 100}}
0.961492 (0.006157) with: {'class_weight': {0: 30, 1: 100}}
0.965406 (0.006037) with: {'class_weight': {0: 40, 1: 100}}
0.970746 (0.005015) with: {'class_weight': {0: 50, 1: 100}}
0.972881 (0.005684) with: {'class_weight': {0: 60, 1: 100}}
0.975309 (0.005711) with: {'class_weight': {0: 70, 1: 100}}
0.975118 (0.004984) with: {'class_weight': {0: 80, 1: 100}}
0.974037 (0.004389) with: {'class_weight': {0: 90, 1: 100}}
0.975920 (0.004870) with: {'class_weight': {0: 100, 1: 100}}


In [8]:
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)

# Save the DataFrame to an Excel file
results_df.to_excel('DT-telco3-ctgan-gmean-fn.xlsx', index=False)